# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Matharu  ->  J. Matharu  |  ['J. Matharu']


J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']
Arxiv has 49 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.16766
extracting tarball to tmp_2602.16766...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.16778
extracting tarball to tmp_2602.16778... done.
Retrieving document from  https://arxiv.org/e-print/2602.17177


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.16766/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'rej_dwarfs_latex' from 'tmp_2602.16766/rej_dwarfs_latex.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'in_gaps_table_latex' from 'tmp_2602.16766/in_gaps_table_latex.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/py

extracting tarball to tmp_2602.17177... done.


...

Retrieving document from  https://arxiv.org/e-print/2602.17593
extracting tarball to tmp_2602.17593...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.17593/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_sample' from 'tmp_2602.17593/table_sample.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']


Issues with the citations
repeated bibliography entry: Prochaska_2019


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17177-b31b1b.svg)](https://arxiv.org/abs/2602.17177) | **PRODIGE - envelope to disk with NOEMA: VII. (Complex) organic molecules in the NGC1333 IRAS4B1 outflow: A new laboratory for shock chemistry**  |
|| L. A. Busch, et al. -- incl., <mark>C. Gieser</mark>, <mark>T. Henning</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Shock chemistry is an excellent tool to shed light on the formation and destruction mechanisms of complex organic molecules (COMs). The L1157-mm outflow is the only low-mass protostellar outflow that has extensively been studied in this regard. Using the data taken as part of the PRODIGE (PROtostars & DIsks: Global Evolution) large program, we aim to map COM emission and derive the molecular composition of the protostellar outflow driven by the Class 0 protostar NGC1333 IRAS4B1 to introduce it as a new laboratory to study the impact of shocks on COM chemistry. In addition to typical outflow tracers such as SiO and CO, outflow emission is seen from H2CO, HNCO, and HC3N, as well as from the COMs CH3OH, CH3CN, and CH3CHO, and even from deuterated species such as DCN, D2CO, and CH2DOH. Maps of integrated intensity ratios between CH3OH and DCN, D2CO, and CH3CHO reveal gradients with distance from the protostar. Intensity ratio maps of HC3N and CH3CN with respect to CH3OH peak in the southern lobe where temperatures are highest. Rotational temperatures derived towards two positions, one in each lobe, are found in the range ~50-100 K. Abundances with respect to CH3OH are higher by factors of a few than for the L1157-B1. In conclusion, for the first time, we securely detected the COMs CH3CN, CH3CHO, and CH2DOH in the IRAS 4B1 outflow, serendipitously with limited sensitivity and bandwidth. Targeted observations will enable the discovery of new COMs and a more detailed analysis of their emission. Morphological differences between molecules in the IRAS 4B1 outflow lobes and their relative abundances provide first proof that this outflow is a promising new laboratory for shock chemistry, which will offer crucial information on COM formation and destruction as well as outflow structure and kinematics.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17329-b31b1b.svg)](https://arxiv.org/abs/2602.17329) | **MIDIS: The identification of deep MIRI-red sources as candidates for extreme Balmer-break and line emitting galaxies at high-z**  |
|| I. Jermann, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *19 pages, 23 figures, submitted to A&A*|
|**Abstract**|            We investigate the detection and nature of 5.6~\mu m MIRI-red sources in the MIRI Deep Imaging Survey (MIDIS), covering 2.4~arcmin$^2$ in the Hubble Ultra Deep Field. MIDIS is the deepest JWST/MIRI survey to date, probing faint limits and enabling studies of rare high-redshift galaxy populations. We define MIRI-red sources as those detected at 5$\sigma$ significance in MIRI/F560W with red colors: $m_{\rm F444W} - m_{\rm F560W} \ge 0.5$. Using an empirical methodology, we estimate the purity and completeness of MIRI detections and find that a 5-sigma detection at 28.75 mag has a purity of 92\% and completeness of 54\%. We identify seven MIRI-red galaxy candidates, including an F115W dropout consistent with a high-redshift galaxy candidate. We explore possible physical origins for the MIRI-red population, including active galactic nuclei, dust-obscured galaxies, extreme emission-line galaxies, evolved stellar populations, and Little Red Dots (LRDs). Given the proximity of the F444W and F560W filters and the depth of MIDIS, MIRI-red galaxies are consistent with emission-line galaxies with $EW_0(H\alpha) \ge 750$ Å or $EW_0(H\beta + [OIII]) \ge 600$ Å, or high-redshift Balmer breaks of at least 1.6. We also discuss an extreme MIRI-red galaxy undetected in F444W, a potential MIRI-only source, for which we derive $EW_0(H\alpha) \sim 6000$ Å and $EW_0(H\beta + [OIII]) \sim 4000$ Å, or high-$z$ LRD analogs with Balmer breaks of 6.3. Finally, we find fewer MIRI-red detections than expected from extrapolations of the H$\alpha$ or H$\beta$+[OIII] line luminosity functions, consistent with previous deep searches, while the absence of $z>10$ LRD candidates agrees with theoretical expectations for the MIDIS volume.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17499-b31b1b.svg)](https://arxiv.org/abs/2602.17499) | **UV slopes of Starforming Galaxies in Strong Lensing fields at the Epoch of Reionization with JWST**  |
|| G. Felicioni, et al. -- incl., <mark>J. Matharu</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *9 pages, 6 figures, 2 tables, submitted to A&A*|
|**Abstract**|            UV slopes ($\beta$) are a powerful diagnostics for galaxies at the Epoch of Reionization, tracing star formation, ISM ionization, and the escape fraction $f_{esc}$ of ionizing photons. Studies at low and intermediate z find a gradual $\beta$ reddening with time and steeper slopes for fainter galaxies, however recent JWST studies reveal a flattening of this trend at $z>7$. We want measure $\beta$ for galaxies at $z>7.5$ using the strong lensing around massive galaxy clusters to observe high-redshift and faint galaxies. The low-brightness regime is of particular interest for reionization, as most of the recent models of this process posit that numerous faint galaxies are the prime drivers of reionization. We use NIRCam and NIRSpec data from CANUCS, Technicolor, JUMPS, Silver Bullet, UNCOVER and MEGASCIENCE across 7 strong lensing fields. We find galaxies down to $M_{UV}\sim-16$ and 7.5<z<12.5. We measure \b{eta} with a forward-modelling procedure and estimate $f_{esc}$ for a subsample with emission line data using a relation, calibrated from a low-z sample, with UV slope, galaxy size and H$\beta$ equivalent width. We find 378 galaxies (45 with spectrum), yielding average values $\beta=-2.3\pm0.4$, $z=8.5\pm1.0$, and $M_{UV}=-18\pm1$. We find no significant $\beta$ evolution across our redshift range, suggesting a flattening of the $\beta-z$ trend above $z\sim7.5$. We find a weak negative trend between $\beta$ and $M_{UV}$. For 14 galaxies we estimate an average $f_{esc}=0.26\pm0.22$. The flat trend of $\beta$ at $z>7.5$ suggests similar properties between $300$ and $600 Myr$ after the Big Bang. The weak trend between $\beta$ and $M_{UV}$ suggests an analogous composition for low- and high-mass galaxies' ISM, likely due to a lack of time for dust buildup. While average $f_{esc}$ is higher than necessary to ionize the IGM by z~6, the model extrapolated at low-z may overestimate its value.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17593-b31b1b.svg)](https://arxiv.org/abs/2602.17593) | **MUSEQuBES: Probing Anisotropies in Gas and Metal Distributions in the Circumgalactic Medium**  |
|| S. Dutta, et al. -- incl., <mark>J. Braspenning</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *Submitted to ApJ. Comments are welcome*|
|**Abstract**|            We investigate the azimuthal dependence of H I and O VI-bearing gas in the circumgalactic medium (CGM) of 113 isolated galaxies in the redshift range 0.12 < z < 0.75, including 91 new measurements from the MUSE Quasar-fields Blind Emitters Survey (MUSEQuBES). The H I covering fraction (k_HI) within the virial radius (Rvir) of low-mass (7 < log10(M*/Msun)< 9) galaxies, for a threshold column density of log10(N(HI)/cm^-2) = 14.5, exhibits an enhancement along both the disk plane (azimuthal angle phi < 20 degree) and in the polar direction (phi > 70 degree). In contrast, such a bimodal distribution is not observed for higher mass galaxies (9 < log10(M*/Msun) < 11.3). Similarly, the O VI covering fraction (k_OVI), for a threshold of log10(N(OVI)/cm^-2) = 14.0, shows a tentative enhancement along both the projected major and minor axes for low-mass galaxies. In contrast, O VI-bearing gas around higher- mass galaxies appears more uniformly distributed, with no significant azimuthal dependence. Finally, using the halo circular-velocity-normalized pixel-velocity two-point correlation function (TPCF), we find that O VI absorbers are kinematically narrower along the disk plane compared to the polar directions of the host galaxies with similar stellar mass distributions. The observed isotropic distribution of O VI in high-mass halos suggests that its spatial distribution is governed by global halo properties; however, the O VI kinematics retain memory of the site of origin.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16766-b31b1b.svg)](https://arxiv.org/abs/2602.16766) | **A Sample of Nearby Isolated Dwarf Galaxies: A First Look at the Mass Function of Field Dwarfs**  |
|| S. Carlsten, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *Submitted to ApJ and posted after responding to referee report. Comments welcome*|
|**Abstract**|            We present the results of the Exploration of Local VolumE Survey - Field (ELVES-Field), a survey of the dwarf galaxies in the Local Volume (LV; $D<10$ Mpc) over roughly $3,000$ square degrees, focusing on the field dwarf population. Candidates are detected using a semi-automated algorithm tailored for low surface brightness dwarfs. Using tests with injected galaxies, we show the detection is $50\%$ complete to $m_g\sim20$ mag and $M_\star \sim 10^6$ $M_\odot$. Candidates are confirmed to be true nearby dwarfs through distance measurements including redshift, tip of the red giant branch, and surface brightness fluctuations. We identify isolated, field dwarfs using various environmental criteria. Over the survey footprint, we detect and confirm 95 LV dwarfs, 44 of which we consider isolated. Using this sample, we infer the field dwarf mass function and find good agreement at the high-mass end with previous redshift surveys and with the predictions of the IllustrisTNG simulation. This sample of isolated, field dwarfs represents a powerful dataset to investigate aspects of small-scale structure and the effect of environment on dwarf galaxy evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16778-b31b1b.svg)](https://arxiv.org/abs/2602.16778) | **ELVES-Field: Isolated Dwarf Galaxy Quenched Fractions Rise Below $M_* \approx 10^7$ $M_\odot$**  |
|| S. Carlsten, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *Accepted to ApJL. Comments welcome*|
|**Abstract**|            We use a new sample of low-mass ($M_* < 10^9$ $M_\odot$) isolated galaxies from the Exploration of Local VolumE Survey - Field (ELVES-Field) to examine the star formation properties and sizes of field dwarf galaxies in the Local Volume (LV; $D<10$ Mpc). This volume-limited sample was selected from nearly 3,000 square degrees of imaging, relying on surface brightness fluctuations to determine distances to the majority of the systems and is complete to $M_* \approx 10^6$ $M_\odot$. Across the surveyed area, we catalog over 2300 candidate LV dwarfs, of which we confirm 95 as genuine LV members and reject over 1600 as background contaminants, with the remaining 600 candidates still requiring a distance measurement. Of the confirmed LV dwarfs, 46 are either new discoveries or confirmed via a distance measurement for the first time here. We explore different environmental criteria to select isolated dwarfs but primarily focus on dwarfs that are $>2\times R_{\mathrm{vir}}$ in projection from any known group with $M_\star > 10^9$ $M_\odot$. We find that, at higher dwarf masses ($M_\star \gtrsim 10^7$ $M_\odot$), essentially all field dwarfs are star-forming as has been found before. In contrast, at $M_\star \lesssim 10^7$ $M_\odot$, $\sim30\%$ of field dwarfs appear to be quenched. Finally, we find that isolated dwarfs are noticeably smaller ($\sim 20\%$) than satellite dwarfs of the same stellar mass, regardless of quenched status.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17267-b31b1b.svg)](https://arxiv.org/abs/2602.17267) | **Illuminating the Mass Gap Through Deep Optical Constraint on a Neutron Star Merger Candidate S250206dm**  |
|| Z. Liu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *20 pages, 10 figures, 6 tables, accepted by ApJL*|
|**Abstract**|            The gravitational wave (GW) event S250206dm, as the first well-localized neutron star merger candidate potentially located in the mass gap, presented a unique opportunity to probe the electromagnetic signatures from such a system. Here we report a deep, multiband search with the new 2.5-meter Wide Field Survey Telescope (WFST), covering about 64% of the localization region up to a 5-sigma limiting magnitude of 23 mag. In total, 12 potential candidates have been identified while none of them are likely related to S250206dm. This non-detection provides the most stringent constraint to date on any associated kilonova. Crucially, an AT 2017gfo-like event at 269 Mpc can be excluded by WFST observations alone. Based on ejecta mass limits, a neutron star-black hole with a large mass ratio (Q >= 3.2) is disfavored. This optical-derived constraint on the mass ratio reaches, for the first time, a precision comparable to that inferred from the GW signal. This work presents the best observation of this type of events until now, and demonstrates the power of rapid, deep follow-up observations to constrain the properties of compact binary progenitors, offering key insights into the constituents of the mass gap.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.17205-b31b1b.svg)](https://arxiv.org/abs/2602.17205) | **Deeper detection limits in astronomical imaging using self-supervised spatiotemporal denoising**  |
|| Y. Guo, et al. -- incl., <mark>Y. Wu</mark> |
|*Appeared on*| *2026-02-20*|
|*Comments*| *Published in Science. This is the author's version of the work. It is posted here by permission of the AAAS for personal use, not for redistribution*|
|**Abstract**|            The detection limit of astronomical imaging observations is limited by several noise sources. Some of that noise is correlated between neighbouring image pixels and exposures, so in principle could be learned and corrected. We present an astronomical self-supervised transformer-based denoising algorithm (ASTERIS), that integrates spatiotemporal information across multiple exposures. Benchmarking on mock data indicates that ASTERIS improves detection limits by 1.0 magnitude at 90% completeness and purity, while preserving the point spread function and photometric accuracy. Observational validation using data from the James Webb Space Telescope (JWST) and Subaru telescope identifies previously undetectable features, including low-surface-brightness galaxy structures and gravitationally-lensed arcs. Applied to deep JWST images, ASTERIS identifies three times more redshift > 9 galaxy candidates, with rest-frame ultraviolet luminosity 1.0 magnitude fainter, than previous methods.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.17177/./figs/maps/H2CO_intensity_map.png', 'tmp_2602.17177/./figs/maps/CH3OH_intensity_map.png', 'tmp_2602.17177/./figs/maps/CH3CHO_intensity_map.png', 'tmp_2602.17177/./figs/maps/D2CO_intensity_map.png', 'tmp_2602.17177/./figs/maps/DCN_intensity_map.png', 'tmp_2602.17177/./figs/maps/c-C3H2_intensity_map.png', 'tmp_2602.17177/./figs/maps/HC3N_intensity_map.png', 'tmp_2602.17177/./figs/maps/CH3CN_intensity_map.png', 'tmp_2602.17177/./figs/maps/HNCO_intensity_map.png', 'tmp_2602.17177/./figs/maps/mom0_main.png', 'tmp_2602.17177/./figs/mom0_ratios.png']
copying  tmp_2602.17177/./figs/maps/H2CO_intensity_map.png to _build/html/
copying  tmp_2602.17177/./figs/maps/CH3OH_intensity_map.png to _build/html/
copying  tmp_2602.17177/./figs/maps/CH3CHO_intensity_map.png to _build/html/
copying  tmp_2602.17177/./figs/maps/D2CO_intensity_map.png to _build/html/
copying  tmp_2602.17177/./figs/maps/DCN_intensity_map.png to _build/html/
copying  tmp_2602.17177/./figs/maps/c-C3H

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mmc}{CH_3SH\xspace}$
$\newcommand{\met}{CH_3OH\xspace}$
$\newcommand{\metc}{CH_3CN\xspace}$
$\newcommand{\metctr}{CH_3^{13}CN\xspace}$
$\newcommand{\et}{C_2H_5OH\xspace}$
$\newcommand{\etc}{C_2H_5CN\xspace}$
$\newcommand{\vc}{C_2H_3CN\xspace}$
$\newcommand{\dme}{CH_3OCH_3\xspace}$
$\newcommand{\mf}{CH_3OCHO\xspace}$
$\newcommand{\mic}{CH_3NCO\xspace}$
$\newcommand{\ad}{CH_3CHO\xspace}$
$\newcommand{\fmm}{NH_2CHO\xspace}$
$\newcommand{\2}{_2}$
$\newcommand{\3}{_3}$
$\newcommand{\5}{_5}$
$\newcommand{\7}{_7}$
$\newcommand{\ct}{^{13}}$
$\newcommand{\cd}{^{12}}$
$\newcommand{\ratio}{^{12}C / ^{13}C\xspace}$
$\newcommand{\oratio}{^{16}O / ^{18}O\xspace}$
$\newcommand{\odh}{^{18}}$
$\newcommand{\scm}{cm^{-2}\xspace}$
$\newcommand{\kms}{ km s^{-1}\xspace}$</div>



<div id="title">

# PRODIGE -- envelope to disk with NOEMA: VII. (Complex) organic molecules in the NGC1333 IRAS 4B1 outflow: \ A new laboratory for shock chemistry

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.17177-b31b1b.svg)](https://arxiv.org/abs/2602.17177)<mark>Appeared on: 2026-02-20</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

L. A. Busch, et al. -- incl., <mark>C. Gieser</mark>, <mark>T. Henning</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Shock chemistry is an excellent tool to shed light on the formation and destruction mechanisms of complex organic molecules (COMs). The L1157-mm outflow is the only low-mass protostellar outflow that has extensively been studied in this regard. We aim to map COM emission and derive the molecular composition of the protostellar outflow driven by the Class 0 protostar NGC 1333 IRAS 4B1 to introduce it as a new laboratory to study the impact of shocks on COM chemistry. We used the data taken as part of the PRODIGE (PROtostars \& DIsks: Global Evolution) large program to compute integrated intensity maps of outflow emission to identify spatial differences between species. The emission spectra were then analysed towards two positions, one in each outflow lobe, by deriving synthetic spectra and population diagrams assuming conditions of local thermodynamic equilibrium (LTE). In addition to typical outflow tracers such as SiO and CO, outflow emission is seen from H \2 CO, HNCO, and HC \3 N, as well as from the COMs $\met$ , $\metc$ , and $\ad$ , and even from deuterated species such as DCN, D \2 CO, and CH \2 DOH. Maps of integrated intensity ratios between $\met$ and DCN, D \2 CO, and $\ad$ reveal gradients with distance from the protostar. For DCN and D \2 CO, this may reflect their prestellar abundance profile, provided the outflow is young enough, while an explanation is  still required for $\ad$ .   Intensity ratio maps of HC \3 N and $\metc$ with respect to $\met$ peak in the southern lobe where temperatures are highest. This may point to enhanced HC \3 N and $\metc$ formation at this location, potentially in the warmer gas phase.   Rotational temperatures are found in the range $\sim$ 50 --100 K, which is, on average, warmer than for the L1157-B1 shock spot ( $\lesssim$ 30 K). Abundances with respect to $\met$ are higher by factors of a few than for L1157-B1. For the first time, we securely detected COMs $\metc$ , $\ad$ , and CH \2 DOH in the IRAS 4B1 outflow, serendipitously with limited sensitivity and bandwidth. Targeted observations will enable the discovery of new COMs and a more detailed analysis of their emission. Morphological differences between molecules in the IRAS 4B1 outflow lobes and their relative abundances provide first proof that this outflow is a promising new laboratory for shock chemistry, which will offer crucial information on COM formation and destruction as well as outflow structure and kinematics.

</div>

<div id="div_fig1">

<img src="tmp_2602.17177/./figs/maps/H2CO_intensity_map.png" alt="Fig11.1" width="11%"/><img src="tmp_2602.17177/./figs/maps/CH3OH_intensity_map.png" alt="Fig11.2" width="11%"/><img src="tmp_2602.17177/./figs/maps/CH3CHO_intensity_map.png" alt="Fig11.3" width="11%"/><img src="tmp_2602.17177/./figs/maps/D2CO_intensity_map.png" alt="Fig11.4" width="11%"/><img src="tmp_2602.17177/./figs/maps/DCN_intensity_map.png" alt="Fig11.5" width="11%"/><img src="tmp_2602.17177/./figs/maps/c-C3H2_intensity_map.png" alt="Fig11.6" width="11%"/><img src="tmp_2602.17177/./figs/maps/HC3N_intensity_map.png" alt="Fig11.7" width="11%"/><img src="tmp_2602.17177/./figs/maps/CH3CN_intensity_map.png" alt="Fig11.8" width="11%"/><img src="tmp_2602.17177/./figs/maps/HNCO_intensity_map.png" alt="Fig11.9" width="11%"/>

**Figure 11. -** Integrated intensity maps towards IRAS 4B1 and 4B2 (green stars) for the molecular transitions that are written on top with their respective upper-level energy. Contours are at $-$5$\sigma$ 5$\sigma$, 10$\sigma$, and then increase by a factor 3 for all but SiO, CO, H\2CO, and $\met$, for which contours start at $-$10$\sigma$ and 10$\sigma$. The noise levels, $\sigma$, are measured in the respective maps and are written in the bottom right. Velocity (in $\kms$) ranges for integration are indicated in the top left. The grey scale shows the sum of the red- and blueshifted maps. The HPBW is shown in the bottom left. The dashed grey circle indicates the primary beam ($\sim$23$\arcsec$). Positions R1 and B1 (green circles) are selected for further spectral-line analysis. (*fig:mom0*)

</div>
<div id="div_fig2">

<img src="tmp_2602.17177/./figs/maps/mom0_main.png" alt="Fig1" width="100%"/>

**Figure 1. -** Integrated intensity maps towards IRAS 4B1 (yellow star) for SiO 2 -- 1 ($E_u=31$ K; colour) and $\met$$4_2-3_1$ and DCN 3 -- 2 ($E_u=45$ K and 21 K, respectively; contours). The SiO maps are integrated from $-25$ to 6 $\kms$ and 6.8 to 47 $\kms$ and show emission above 10$\sigma$ with $\sigma_\mathrm{blue}=0.42$ K $\kms$ and $\sigma_\mathrm{red}=0.49$ K $\kms$. Contours are at $-$30$\sigma$,$-$15$\sigma$, 15$\sigma$, 30$\sigma$, and then increase by a factor of 3 for $\met$($\sigma_\mathrm{blue}=0.25$ K $\kms$ and $\sigma_\mathrm{red}=0.23$ K $\kms$) and at $-$10$\sigma$, $-$5$\sigma$, 5$\sigma$, 10$\sigma$, and then increase by a factor of 3 for DCN ($\sigma_\mathrm{blue}=0.17$ K $\kms$ and $\sigma_\mathrm{red}=0.24$ K $\kms$). Velocity ranges (in $\kms$) used for the integration of $\met$ and DCN emission are indicated in the top left. The HPBW is shown in the bottom left. Positions R1 and B1 were selected for further spectral-line analysis (Sect. \ref{ss:analysis}). Black crosses indicate H\2 knots \citep[][]{Choi2011}. (*fig:main*)

</div>
<div id="div_fig3">

<img src="tmp_2602.17177/./figs/mom0_ratios.png" alt="Fig2" width="100%"/>

**Figure 2. -** Integrated intensity ratios, $(W_\mathrm{mol} / W_\mathrm{CH_3OH})$ / $( W_\mathrm{mol} / W_\mathrm{CH_3OH})_\mathrm{max}$ of various molecular transitions with either $\met$$10_{2,9}-9_{3,6}$($E_u=165$ K; for HC\3N and $\met$c) or $5_{1,4}-4_{2,2}$($E_u=56$ K; for D\2CO, DCN, HNCO, and $\ad$) towards the IRAS 4B1 outflow. Integrated intensities contain the sum of blue- and redshifted emission (cf. Fig. \ref{fig:mom0} and spectra in Fig. \ref{fig:spectra}). The ratio is only shown if both molecules are above a 5$\sigma$ threshold, where $\sigma$ is the noise level in the respective map. Another mask with a 1$\arcsec$  radius around the protostar is applied to the ratios.
    Black contours show the integrated intensities of $\met$, starting at $-5\sigma$, 5$\sigma$, and then increasing by a factor of 2, where $\sigma=0.34$ K $\kms$. The molecule as well as the upper-level energy of the shown transition are in the top left corner. In all panels, the white star marks the position of IRAS 4B1. The HPBW is shown in the bottom left corner and black crosses mark H\2 knots  ([Choi and Lee 2011](https://ui.adsabs.harvard.edu/abs/2011JKAS...44..201C)) . Spectroscopic information on the shown transitions are given in Table \ref{tab:lines}. (*fig:ratios*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.17177"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{{\color{red}#1}}$
$\newcommand{\blue}[1]{{\color{blue}#1}}$
$\newcommand{\purple}[1]{{\color{purple}#1}}$
$\newcommand{\pink}[1]{{\color{pink}#1}}$
$\newcommand{\green}[1]{{\color{green}#1}}$
$\newcommand{\arcsecs}{\mbox{^{\prime\prime}}}$
$\newcommand{\Msolar}{\mbox{\rm M_{\odot} }}$
$\newcommand{\orcid}[1]{\includegraphics[scale=0.06]{Figures/ORCID-iD_icon-128x128.png} \href{https://orcid.org/#1}{#1}}$
$\newcommand{\gs}{\mathrel{\raise0.35ex\hbox{\scriptstyle >}\kern-0.6em \lower0.40ex\hbox{{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\raise0.35ex\hbox{\scriptstyle <}\kern-0.6em \lower0.40ex\hbox{{\scriptstyle \sim}}}}$</div>



<div id="title">

# MIDIS: The identification of deep MIRI-red sources as candidates for extreme Balmer-break and line emitting galaxies at high-$z$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.17329-b31b1b.svg)](https://arxiv.org/abs/2602.17329)<mark>Appeared on: 2026-02-20</mark> -  _19 pages, 23 figures, submitted to A&A_

</div>
<div id="authors">

I. Jermann, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We investigate the detection and nature of 5.6 $\mu$ m MIRI-red sources in the MIRI Deep Imaging Survey (MIDIS), covering 2.4 arcmin $^2$ in the Hubble Ultra Deep Field. MIDIS is the deepest _JWST_ /MIRI survey to date, probing the faintest limits and enabling the study of rare galaxy populations at high redshift. We define MIRI-red sources as detected at $5\sigma$ significance in MIRI/F560W with red colors: $m_{\rm F444W}-m_{\rm F560W} \geq 0.5$ . We characterize the MIDIS background to assess the effective depth and noise properties of faint MIRI detections, finding that pipeline uncertainties lead to a systematic overestimation of the signal-to-noise ratio by a factor of $\sim 3$ . Using an empirical methodology, we estimate the purity and completeness of MIRI detections and find that a $5\sigma$ detection at 28.75 mag has a purity of 92 \% and a completeness of 54 \% . We identify seven MIRI-red galaxy candidates, including an F115W dropout consistent with a high-redshift galaxy candidate. We explore possible physical origins for the MIRI-red population, including active galactic nuclei, dust-obscured galaxies, extreme emission-line galaxies, evolved stellar populations, and Little Red Dots (LRDs). Given the proximity of the F444W and F560W filters and the depth of MIDIS, we find that MIRI-red galaxy candidates are consistent with either emission-line galaxies with $EW_0({\rm H\alpha})\geq 750$ Å or $EW_0{(\rm H\beta+[OIII])}\geq 600$ Å, or high-redshift Balmer breaks of at least 1.6. We additionally discuss the extreme case of a MIRI-red galaxy candidates undetected in F444W, a potential MIRI-only source, for which we derive extreme line strengths of $EW_0({\rm H\alpha})\sim6000$ Å and $EW_0({\rm H\beta+[OIII]}) \sim 4000$ Å, or high- $z$ LRD analogs exhibiting Balmer breaks of 6.3. Finally, we find fewer MIRI-red detections than expected from extrapolations of the H $\alpha$ or H $\beta$ + [ OIII ] emitter line luminosity functions, consistent with previous deep searches, while the absence of $z>10$ LRD candidates is consistent with theoretical expectations for the MIDIS survey volume.

</div>

<div id="div_fig1">

<img src="tmp_2602.17329/./Figures/PAPER_MIRIonlyFigure7_ColorPlot.png" alt="Fig11" width="100%"/>

**Figure 11. -** NIRCam/F444W$-$MIRI/F560W colors of MIRI detections in the MIDIS Deep survey with F560W magnitudes $>23.5$ mag, shown as a function of F560W magnitude in the left panel and spectroscopic redshift in the right panel. The red and golden shaded region indicates the color space occupied by MIRI-red and potential MIRI-only sources. (*fig:ColorPlot*)

</div>
<div id="div_fig2">

<img src="tmp_2602.17329/./Figures/PAPER_MIRIonly_FigureSEDdeep_final.png" alt="Fig10" width="100%"/>

**Figure 10. -**  Eazy SED fit of the MIRI-red galaxy candidates sample including the deep F770W and F1000W data from MIDIS-red. In Appendix \ref{sec:AppendixD_SED}, we show the same figure without MIDIS-red data. Interestingly, without the MIDIS-red data, MIRI-red candidate with ID=205 shows H$\alpha$ emission line in F560W instead of F770W, as it is not constrained without the MIDIS-red observations. In Appendix \ref{sec:AppendixD_SED} are the photometric redshift probability distribution corresponding to all the fits.  (*fig:MIRIonly_red_SED*)

</div>
<div id="div_fig3">

<img src="tmp_2602.17329/./Figures/PAPER_MIRIonlyFigure_PurityCompleteness.png" alt="Fig4" width="100%"/>

**Figure 4. -**  Purity and completeness of the MIDIS Deep survey (excluding detection brighter than 23.5 mag) as a function of MIRI 5.6 $\mu$m magnitude and flux. At a $5\sigma$ detection threshold, the survey reaches 92\% purity and 54\% completeness.
    (*fig:MIRIpurity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.17329"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\angstrom}{\textup{Å}}$</div>



<div id="title">

# UV slopes of Starforming Galaxies in Strong Lensing fields at the Epoch of Reionization with JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.17499-b31b1b.svg)](https://arxiv.org/abs/2602.17499)<mark>Appeared on: 2026-02-20</mark> -  _9 pages, 6 figures, 2 tables, submitted to A&A_

</div>
<div id="authors">

G. Felicioni, et al. -- incl., <mark>J. Matharu</mark>

</div>
<div id="abstract">

**Abstract:** UV slopes ( $\beta$ ) are a powerful diagnostics for galaxies at the epoch of reionization, tracing star formation, ISM ionization, and the escape fraction $f_{esc}$ of ionizing photons. Studies at low and intermediate $z$ find suggest a gradual reddening of the UV slope with time and steeper slopes for fainter galaxies. With the James Webb Space Telescope ( _JWST_ ), it has been possible to measure the UV slope for statistically significant samples of galaxies deep into the EoR, up to redshift $z\gtrsim10$ and for increasingly faint galaxies. Recent studies of $\beta$ with JWST at $z>7$ reveal a flattening of the trend of the UV slope with respect to redshift and UV magnitude. We want to measure $\beta$ at $z>7.5$ using the strong lensing around massive galaxy clusters to observe high-redshift and faint galaxies. The low-brightness regime is of particular interest for reionization, as most of the recent models of this process posit that numerous faint galaxies are the prime drivers of reionization We use NIRCam and NIRSpec observations from CANUCS, Technicolor, JUMPS, Silver Bullet, UNCOVER and MEGASCIENCE across 7 strong lensing fields in order to find galaxies down to $M_{UV}\sim-16$ and $7.5<z\lesssim12.5$ . We measure their $\beta$ with a forward-modelling procedure. We estimate the $f_{esc}$ of a subsample with emission line data using a relation with UV slope, galaxy size and H $\beta$ equivalent width (EWH $\beta$ ) calibrated from a low-redshift sample. We find 378 visually confirmed galaxies, including 45 with a spectrum. We obtain an average UV slope $<\beta>=-2.3\pm0.4$ , an average redshift $<z>=8.5\pm1.0$ , and an average AB magnitude $M_{UV}=-18\pm1$ . We find no significant evolution of $\beta$ across our redshift range, suggesting a flattening of the $\beta-z$ trend above $z\sim7.5$ . We find a weak trend between $\beta$ and $M_{UV}$ . For a subsample of 14 galaxies with emission line data we estimate an average $f_{esc}=0.26\pm0.22$ The flat trend of $\beta$ at $z>7.5$ indicates a  similarity in average galaxy properties between $300$ and $600 Myr$ after the Big Bang. The weak trend between $\beta$ and $M_{UV}$ suggests an analogous composition for low- and high-mass galaxies' ISM, likely due to a lack of time to build up a dust reservoir. The average $f_{esc}$ is high for what is necessary to achieve an ionized IGM by $z\sim6$ , but the extrapolation model from a low-redshift sample may be overestimating its value.

</div>

<div id="div_fig1">

<img src="tmp_2602.17499/./Images/canucs+bullet+uncover_140_to_300nm_MCMC_beta_z_fit_v2.png" alt="Fig4" width="100%"/>

**Figure 4. -** $\beta-z$ diagram for our high-redshift sample. Objects for which only photometric estimates of $\beta$ are available are plotted in blue, and spectral estimates of $\beta$ are in orange; for objects that have both we only plot spectral estimates. A linear fit of the trend and its shaded 1-$\sigma$ confidence level, obtained via a bootstrap method, is also represented. (*fig:beta-z*)

</div>
<div id="div_fig2">

<img src="tmp_2602.17499/./Images/4117358_cutouts.png" alt="Fig1.1" width="33%"/><img src="tmp_2602.17499/./Images/4117358_beta_plot.png" alt="Fig1.2" width="33%"/><img src="tmp_2602.17499/./Images/4117358_NIRSpec.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** _Top panel_: NIRCam and ACS images of an example object (CANUCS ID 4117358) in the MACS1423 cluster in the CLU pointing, in all of the filters available for this pointing.\ _Middle_: NIRCam photometry overlaid with the throughput of each NIRCam filter, with the fitted $\beta$ slope in yellow. The JWST filters' throughput functions are also overlaid in different colours. Vertical dashed (dotted) lines represent the lower (upper) limit for the filters to be used. A hard limit for the lower wavelength threshold, and a soft limit, only limiting the filter's pivot wavelength, for the upper threshold, were applied; in this case, the F150W, F200W and F277W filters were used to fit the UV slope.\ _Bottom panel:_ 2D and 1D spectrum for the same object. In the 1D spectrum, the gray line represents the signal, while the red line represents the noise at each wavelength of the spectrum. (*fig:placeholder*)

</div>
<div id="div_fig3">

<img src="tmp_2602.17499/./Images/3211032_beta_plot.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example of a galaxy at $z=7.87$ from the NCF pointing of the MACS0416 field, whose $\beta$ was calculated from 8 NIRCam filters: F140M, F150W, F162M, F182M, F200W, F210M, and F250M (*fig:8_filters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.17499"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sqcm}{cm^{-2}}$
$\newcommand{\kms}{\rm km~s^{-1}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyb}{Ly\beta}$
$\newcommand{\lyg}{Ly\gamma}$
$\newcommand{\lyd}{Ly\delta}$
$\newcommand{\HeI}{\mbox{He {\sc i}}}$
$\newcommand{\HeII}{\mbox{He {\sc ii}}}$
$\newcommand{\HI}{\mbox{H {\sc i}}}$
$\newcommand{\Nhi}{N_{\footnotesize{\HI}}}$
$\newcommand{\Nhic}{N_{\footnotesize{\HI}}^{c}}$
$\newcommand{\logNhic}{{\rm log}_{10}(\Nhic/{\rm cm}^{-2})}$
$\newcommand{\logNhi}{{\rm log}_{10}(\Nhi/{\rm cm}^{-2})}$
$\newcommand{\Nhisys}{N_{\footnotesize{\HI}}^{\rm sys}}$
$\newcommand{\HII}{\mbox{H {\sc ii}}}$
$\newcommand{\HH}{\mbox{H{_2}}}$
$\newcommand{\hh}{\mbox{\tiny H{_2}}}$
$\newcommand{\hi}{\mbox{\tiny H{\sc i}}}$
$\newcommand{\OI}{\mbox{O {\sc i}}}$
$\newcommand{\OII}{\mbox{O {\sc ii}}}$
$\newcommand{\OIII}{\mbox{O {\sc iii}}}$
$\newcommand{\OIV}{\mbox{O {\sc iv}}}$
$\newcommand{\OV}{\mbox{O {\sc v}}}$
$\newcommand{\OVI}{\mbox{O {\sc vi}}}$
$\newcommand{\OVII}{\mbox{O {\sc vii}}}$
$\newcommand{\OVIII}{\mbox{O {\sc viii}}}$
$\newcommand{\CaVIII}{\mbox{Ca {\sc viii}}}$
$\newcommand{\CaVII}{\mbox{Ca {\sc vii}}}$
$\newcommand{\CaVI}{\mbox{Ca {\sc vi}}}$
$\newcommand{\CaV}{\mbox{Ca {\sc v}}}$
$\newcommand{\CIV}{\mbox{C {\sc iv}}}$
$\newcommand{\CV}{\mbox{C {\sc v}}}$
$\newcommand{\CVI}{\mbox{C {\sc vi}}}$
$\newcommand{\CII}{\mbox{C {\sc ii}}}$
$\newcommand{\CI}{\mbox{C {\sc i}}}$
$\newcommand{\CIIs}{\mbox{C {\sc ii}}^\ast}$
$\newcommand{\CIII}{\mbox{C {\sc iii}}}$
$\newcommand{\SiI}{\mbox{Si {\sc i}}}$
$\newcommand{\SiII}{\mbox{Si {\sc ii}}}$
$\newcommand{\SiIII}{\mbox{Si {\sc iii}}}$
$\newcommand{\SiIV}{\mbox{Si {\sc iv}}}$
$\newcommand{\SiXII}{\mbox{Si {\sc xii}}}$
$\newcommand{\SV}{\mbox{S {\sc v}}}$
$\newcommand{\SIV}{\mbox{S {\sc iv}}}$
$\newcommand{\SIII}{\mbox{S {\sc iii}}}$
$\newcommand{\SII}{\mbox{S {\sc ii}}}$
$\newcommand{\SI}{\mbox{S {\sc i}}}$
$\newcommand{\ClI}{\mbox{Cl {\sc i}}}$
$\newcommand{\ArI}{\mbox{Ar {\sc i}}}$
$\newcommand{\NI}{\mbox{N {\sc i}}}$
$\newcommand{\NII}{\mbox{N {\sc ii}}}$
$\newcommand{\NIII}{\mbox{N {\sc iii}}}$
$\newcommand{\NIV}{\mbox{N {\sc iv}}}$
$\newcommand{\NV}{\mbox{N {\sc v}}}$
$\newcommand{\PV}{\mbox{P {\sc v}}}$
$\newcommand{\PII}{\mbox{P {\sc ii}}}$
$\newcommand{\PIII}{\mbox{P {\sc iii}}}$
$\newcommand{\PIV}{\mbox{P {\sc iv}}}$
$\newcommand{\NeVIII}{\mbox{Ne {\sc viii}}}$
$\newcommand{\ArVIII}{\mbox{Ar {\sc viii}}}$
$\newcommand{\NeV}{\mbox{Ne {\sc v}}}$
$\newcommand{\NeVI}{\mbox{Ne {\sc vi}}}$
$\newcommand{\NeX}{\mbox{Ne {\sc x}}}$
$\newcommand{\NaIX}{\mbox{Na {\sc ix}}}$
$\newcommand{\MgII}{\mbox{Mg {\sc ii}}}$
$\newcommand{\FeII}{\mbox{Fe {\sc ii}}}$
$\newcommand{\MgX}{\mbox{Mg {\sc x}}}$
$\newcommand{\AlXI}{\mbox{Al {\sc xi}}}$
$\newcommand{\FeIII}{\mbox{Fe {\sc iii}}}$
$\newcommand{\NaI}{\mbox{Na {\sc i}}}$
$\newcommand{\CaII}{\mbox{Ca {\sc ii}}}$
$\newcommand{\zabs}{z_{\rm abs}}$
$\newcommand{\zmin}{z_{\rm min}}$
$\newcommand{\zmax}{z_{\rm max}}$
$\newcommand{\zqso}{z_{\rm qso}}$
$\newcommand{\zlae}{z_{\rm peak}}$
$\newcommand{\zcorr}{z_{\rm corr}}$
$\newcommand{\voff}{V_{\rm offset}}$
$\newcommand{\rew}{W_{r}}$
$\newcommand{\rewf}{W_{r,500}}$
$\newcommand{\rewt}{W_{r,300}}$
$\newcommand{\logm}{{\rm log}_{10}(M_{\star}/\rm M_{\odot})}$
$\newcommand{\Msun}{\rm M_{\odot}}$
$\newcommand{\logn}{{\rm log}_{10}(N/\rm cm^{-2})}$
$\newcommand{\lognovi}{\rm log_{10}(N_{OVI}/cm^{-2})}$
$\newcommand{\loghi}{{\rm log}_{10}(N(\HI)/{\rm cm}^{-2})}$
$\newcommand{\novi}{N_{O\mbox{\sc vi}}}$
$\newcommand{\novie}{N_{O\mbox{\sc vi}}}$
$\newcommand{\movi}{M_{O\mbox{\sc vi}}}$
$\newcommand{\movie}{M_{O\mbox{\sc vi}}}$
$\newcommand{\kHI}{\mbox{\kappa_{\tt H I}}}$
$\newcommand{\kOVI}{\mbox{\kappa_{\tt O VI}}}$
$\newcommand{\blue}{\color{blue}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# MUSEQuBES: Probing Anisotropies in Gas and Metal Distributions in the Circumgalactic Medium

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.17593-b31b1b.svg)](https://arxiv.org/abs/2602.17593)<mark>Appeared on: 2026-02-20</mark> -  _Submitted to ApJ. Comments are welcome_

</div>
<div id="authors">

S. Dutta, et al. -- incl., <mark>J. Braspenning</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ We investigate the azimuthal dependence of $\HI$ and $\OVI$ -bearing gas in the circumgalactic medium (CGM) of 113 isolated galaxies in the redshift range $0.12<z<0.75$ , including 91 new measurements from the MUSE Quasar-fields Blind Emitters Survey (MUSEQuBES).The $\HI$ covering fraction ( $\kHI$ ) within the virial radius ( $R_{\rm vir}$ ) of low-mass ( $7<$ $\logm$  $\leq 9$ ) galaxies, for a threshold column density of ${\rm log}_{10}(N(\HI)/{\rm cm}^{-2})=14.5$ , exhibits an enhancement along both the disk plane (azimuthal angle $\phi\lesssim20^{\circ}$ ) and in the polar direction ( $\phi\gtrsim70^{\circ}$ ). In contrast, such a bimodal distribution is not observed for higher mass galaxies ( $9<$ $\logm$  $\leq 11.3$ ).Similarly, the $\OVI$ covering fraction ( $\kOVI$ ), for a threshold of ${\rm log}_{10}(N(\OVI)/{\rm cm}^{-2})=14.0$ , shows a tentative enhancement along both the projected major and minor axes for low-mass galaxies. In contrast, $\OVI$ -bearing gas around higher-mass galaxies appears more uniformly distributed, with no significant azimuthal dependence.Finally, using the halo circular-velocity-normalized pixel-velocity two-point correlation function (TPCF), we find that $\OVI$ absorbers are kinematically narrower along the disk plane compared to the polar directions of the host galaxies with similar stellar mass distributions.The observed isotropic distribution of $\OVI$ in high-mass halos suggests that its spatial distribution is governed by global halo properties; however, the $\OVI$ kinematics retain memory of the site of origin.

</div>

<div id="div_fig1">

<img src="tmp_2602.17593/./plots2/phi_pdf_OVI_all_invir_wilson_ver.png" alt="Fig15.1" width="33%"/><img src="tmp_2602.17593/./plots2/phi_pdf_OVI_lowm_invir_wilson_ver.png" alt="Fig15.2" width="33%"/><img src="tmp_2602.17593/./plots2/phi_pdf_OVI_highm_invir_wilson_ver.png" alt="Fig15.3" width="33%"/>

**Figure 15. -** Same as Fig. \ref{fig:hi_all} but for $\OV$I. The column density thresholds are $N($\OV$I)=10^{14} {\rm cm}^{-2}$ and $N($\OV$I)=10^{14.3} {\rm cm}^{-2}$ in the {\tt middle} and {\tt bottom} panels, respectively. In {\tt Columns B} and {\tt C}, only star-forming galaxies are selected.  (*fig:ovi_all*)

</div>
<div id="div_fig2">

<img src="tmp_2602.17593/./plots2/phi_pdf_HI_all_invir_wilson_ver.png" alt="Fig14.1" width="33%"/><img src="tmp_2602.17593/./plots2/phi_pdf_HI_lowm_invir_wilson_ver.png" alt="Fig14.2" width="33%"/><img src="tmp_2602.17593/./plots2/phi_pdf_HI_highm_invir_wilson_ver.png" alt="Fig14.3" width="33%"/>

**Figure 14. -** {\tt Column A, Top:} The red solid and open circles show the detected $N($\HI$)$ and $3\sigma$ upper limits for non-detections, respectively, plotted against the most probable $\phi$ for galaxies with $D<R_{\rm vir}$. The red crosses indicate the discarded face-on galaxies with most probable $i<35^{\circ}$. {\tt Column A, Middle:} The $\HI$ covering fraction ($\kHI$) for a threshold $N($\HI$)=10^{14.5} {\rm cm}^{-2}$ plotted against $\phi$ for the galaxy sample shown in the left panel (discarding the face-on galaxies). The shaded and hatched regions represent the 68\% Wilson score interval and 68\% confidence interval obtained from 10000 bootstrap realizations of the galaxy sample, respectively. {\tt Column A, Bottom:} Same as {\tt middle}, but for a threshold $N($\HI$)=10^{15} {\rm cm}^{-2}$. No significant $\phi$ dependence is seen in any of these panels. {\tt Columns B} and {\tt C} is same as {\tt Column A} but for dwarf ($\logm$$\approx7-9$) and massive ($\logm$$\approx 9-11.3$) galaxies, respectively. A marginal reduction in $\kHI$ is seen for the intermediate $\phi$ bin for the dwarf galaxies ({\tt Column B}). However, the $\kHI$ for more massive galaxies does not exhibit any significant azimuthal dependence ({\tt Column C}).  (*fig:hi_all*)

</div>
<div id="div_fig3">

<img src="tmp_2602.17593/./plots2/ovi_tpcf_scaled.png" alt="Fig3" width="100%"/>

**Figure 3. -** The $v_{\rm vir}-$normalized pixel-velocity two-point correlation functions (TPCFs) for $\OV$I absorbers associated with $D<R_{\rm vir}$ galaxies
    for three bins of azimuthal angle as indicated by the legends. The shaded regions represent the 1$\sigma$ confidence intervals obtained from 200 bootstrap realizations. The numbers in the parentheses indicate the number of galaxies contributing to the plot.
    Face-on galaxies with $i<35^{\circ}$ are excluded. (*fig:ovi_kin*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.17593"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

117  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
